# Spectator/TLS example

In [4]:
from c3.experiment import Experiment

## Optimal control simulation

In [7]:
exp = Experiment()
exp.read_config("two_qubits.c3exp")
parameter_map = exp.pmap

2022-08-16 14:24:34.210061: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/ubuntu/c3/c3/signal/gates.py:135: UserWarning: No ideal gate found for gate:  . Use set_ideal() explicitly.
  warnings.warn(


In [8]:
import os
import tempfile
from c3.optimizers.optimalcontrol import OptimalControl
from c3.libraries import algorithms, fidelities

log_dir = os.path.join(tempfile.TemporaryDirectory().name, "c3logs")
opt = OptimalControl(
    dir_path=log_dir,
    fid_func=fidelities.unitary_infid_set,
    fid_subspace=["Q1", "Q2"],
    pmap=parameter_map,
    algorithm=algorithms.lbfgs,
    options={
        "maxfun": 25
    },
    run_name="cnot12"
)
opt_gates = ["cx[0, 1]"]
exp.set_opt_gates(opt_gates)

gateset_opt_map=[
    [("cx[0, 1]", "d1", "gauss1", "amp")],
    [("cx[0, 1]", "d1", "gauss1", "freq_offset")],
    [("cx[0, 1]", "d1", "gauss1", "xy_angle")],
    [("cx[0, 1]", "d1", "gauss1", "delta")],
    [("cx[0, 1]", "d1", "carrier", "framechange")],
    [("cx[0, 1]", "d2", "gauss2", "amp")],
    [("cx[0, 1]", "d2", "gauss2", "freq_offset")],
    [("cx[0, 1]", "d2", "gauss2", "xy_angle")],
    [("cx[0, 1]", "d2", "gauss2", "delta")],
    [("cx[0, 1]", "d2", "carrier", "framechange")],
]
parameter_map.set_opt_map(gateset_opt_map)

parameter_map.print_parameters()
exp.set_opt_gates(opt_gates)
opt.set_exp(exp)
opt.optimize_controls()
parameter_map.print_parameters()
print(opt.current_best_goal)

cx[0, 1]-d1-gauss1-amp                : 800.000 mV 
cx[0, 1]-d1-gauss1-freq_offset        : -53.000 MHz 2pi 
cx[0, 1]-d1-gauss1-xy_angle           : -444.089 arad 
cx[0, 1]-d1-gauss1-delta              : -1.000  
cx[0, 1]-d1-carrier-framechange       : 4.712 rad 
cx[0, 1]-d2-gauss2-amp                : 30.000 mV 
cx[0, 1]-d2-gauss2-freq_offset        : -53.000 MHz 2pi 
cx[0, 1]-d2-gauss2-xy_angle           : -444.089 arad 
cx[0, 1]-d2-gauss2-delta              : -1.000  
cx[0, 1]-d2-carrier-framechange       : 0.000 rad 

C3:STATUS:Saving as: /tmp/tmpd85cbruv/c3logs/cnot12/2022_08_16_T_14_24_36/open_loop.c3log
cx[0, 1]-d1-gauss1-amp                : 2.327 V 
cx[0, 1]-d1-gauss1-freq_offset        : -53.243 MHz 2pi 
cx[0, 1]-d1-gauss1-xy_angle           : -523.431 mrad 
cx[0, 1]-d1-gauss1-delta              : -761.526 m 
cx[0, 1]-d1-carrier-framechange       : -705.444 mrad 
cx[0, 1]-d2-gauss2-amp                : 52.793 mV 
cx[0, 1]-d2-gauss2-freq_offset        : -53.163 MHz 2pi 
cx[0, 

## Calibration

In [16]:
from c3.qiskit.c3_gates import RX90pGate, RX90mGate, RY90pGate, RY90mGate, SetParamsGate
from qiskit import QuantumCircuit
from typing import List
QISKIT_GATE_MAP = { "rx90p": RX90pGate, "rx90m": RX90mGate, "ry90p": RY90pGate, "ry90m": RY90mGate}
def seqs_to_circuit(seqs: List[List[str]]) -> QuantumCircuit:
    circuits = []
    for seq in seqs:
        qc_sec = QuantumCircuit(1, 1)
        for gate in seq:
            qc_sec.append(QISKIT_GATE_MAP[gate[:-3]](), [int(gate[-2])])
        circuits.append(qc_sec)
    return circuits

In [17]:
import numpy as np
import tensorflow as tf
from c3.qiskit import C3Provider
from c3.utils import qt_utils

### ORBIT meta-parameters ###
RB_length = 12 # How long each sequence is
RB_number = 5  # How many sequences
shots = 1000    # How many averages per readout

orbit_provider = C3Provider()
orbit_exp = Experiment()
orbit_exp.read_config("two_qubits_TLS.c3exp")
orbit_backend = orbit_provider.get_backend("c3_qasm_physics_simulator")
orbit_backend.set_c3_experiment(orbit_exp)

def ORBIT_qiskit(params):
    
    populations = []
    results = []
    results_std = []
    shots_nums = []

    # Creating the RB sequences
    seqs = qt_utils.single_length_RB(
            RB_number=RB_number, RB_length=RB_length, target=0
    )
    orbit_exp.set_opt_gates_seq(seqs) # speeds up the simulation of circuits
    circuits = seqs_to_circuit(seqs)

    orbit_job = orbit_backend.run(circuits, params = params, opt_map = gateset_opt_map)
    populations = [list(result.data.state_pops.values()) for result in orbit_job.result().results]
        
    for pop in populations:
        excited_pop = np.array(pop[1:]).sum() # total excited states population
        results.append(np.array([excited_pop]))
        results_std.append([0])
        shots_nums.append([shots])

    goal = np.mean(results) # average of the excited state populations from every circuit
    return goal, results, results_std, seqs, shots_nums

In [18]:
from c3.libraries.algorithms import cmaes
from c3.optimizers.calibration import Calibration
alg_options = {
    "popsize" : 10,
    "maxfevals" : 300,
    "init_point" : "True",
    "tolfun" : 0.01,
    "spread" : 0.1
}
state_labels = {
      "excited" : [(1,), (2,)]
  }
# Create a temporary directory to store logfiles, modify as needed
log_dir = "Entangling_TLS"

opt = Calibration(
    dir_path=log_dir,
    run_name="ORBIT_cal",
    eval_func=ORBIT_qiskit,
    pmap=parameter_map,
    exp_right=exp,
    algorithm=cmaes,
    options=alg_options
)
opt.optimize_controls()

C3:STATUS:Saving as: /home/ubuntu/c3/examples/Entangling_TLS/ORBIT_cal/2022_08_16_T_14_27_45/calibration.log
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=734707, Tue Aug 16 14:27:45 2022)
C3:STATUS:Adding initial point to CMA sample.


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /home/ubuntu/c3/c3/experiment.py:492 in compute_propagators                               │
│                                                                                           │
│   489 │   │                                                                               │
│   490 │   │   for gate in gate_ids:                                                       │
│   491 │   │   │   try:                                                                    │
│ ❱ 492 │   │   │   │   instr = instructions[gate]                                          │
│   493 │   │   │   except KeyError:                                                        │
│   494 │   │   │   │   raise Exception(                                                    │
│   495 │   │   │   │   │   f"C3:Error: Gate '{gate}' is not defined."                      │
╰───────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 'ry90m[0]'

During handling of the above exception, another exception occurred:

╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│                                                                                           │
│ /tmp/ipykernel_2550515/2546843785.py:25 in <cell line: 25>                                │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_2550515/2546843785.py'               │
│ /home/ubuntu/c3/c3/optimizers/calibration.py:103 in optimize_controls                     │
│                                                                                           │
│   100 │   │   print(f"C3:STATUS:Saving as: {os.path.abspath(self.logdir + self.logname)}" │
│   101 │   │   x_init = self.pmap.get_parameters_scaled()                                  │
│   102 │   │   try:                                                                        │
│ ❱ 103 │   │   │   self.algorithm(                                                         │
│   104 │   │   │   │   x_init,                                                             │
│   105 │   │   │   │   fun=self.fct_to_min,                                                │
│   106 │   │   │   │   fun_grad=self.fct_to_min_autograd,                                  │
│                                                                                           │
│ /home/ubuntu/c3/c3/libraries/algorithms.py:555 in cmaes                                   │
│                                                                                           │
│   552 │   │   if batch_noise:                                                             │
│   553 │   │   │   error = np.random.randn() * noise                                       │
│   554 │   │   for sample in samples:                                                      │
│ ❱ 555 │   │   │   goal = fun(sample)                                                      │
│   556 │   │   │   if noise:                                                               │
│   557 │   │   │   │   error = np.random.randn() * noise                                   │
│   558 │   │   │   if batch_noise or noise:                                                │
│                                                                                           │
│ /home/ubuntu/c3/c3/optimizers/optimizer.py:265 in fct_to_min                              │
│                                                                                           │
│   262 │   │   self.optim_status["params"] = pars                                          │
│   263 │   │   if isinstance(input_parameters, np.ndarray):                                │
│   264 │   │   │   current_params = tf.constant(input_